# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU crashes when receiving F1AP Setup Response.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU crashes when receiving F1AP Setup Response or DL RRC Message Transfer due to execution of DU handlers.",
    "interface_guess": [
      "F1AP"
    ],
    "procedure_guess": [
      "DL RRC Message Transfer",
      "F1 Setup",
      "UE Context Modification"
    ],
    "components_involved": [
      "gNB-CU"
    ],
    "key_ids": {
      "rnti": "affected",
      "du_ue_f1ap_id": "affected"
    }
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": [
        "DL RRC Message Transfer",
        "gNB-CU",
        "gNB-DU"
      ],
      "title": [
        "8.4.2 DL RRC Message Transfer"
      ]
    },
    {
      "Keywords": [
        "F1 Setup",
        "Response"
      ],
      "title": [
        "8.5.2 F1 Setup"
      ]
    },
    {
      "Keywords": [
        "UE Context Modification",
        "Initiating Message"
      ],
      "title": [
        "8.3.4 UE Context Modification"
      ]
    }
  ],
  "expected_behaviour": "The gNB-CU initiates DL RRC Message Transfer. The gNB-DU should receive it. The gNB-CU should NOT receive DL RRC Message Transfer. Similarly for F1 Setup Response, it is sent by CU, should not be received by CU."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "openair2/F1AP/f1ap_handlers.c",
      "function_name": "f1ap_handle_message",
      "reason": "Central dispatch table for F1AP messages, lacks node role check."
    },
    {
      "path": "openair2/F1AP/f1ap_du_rrc_message_transfer.c",
      "function_name": "DU_handle_DL_RRC_MESSAGE_TRANSFER",
      "reason": "DU handler for DL RRC Message Transfer, executed by CU."
    },
    {
      "path": "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "function_name": "dl_rrc_message_transfer",
      "reason": "Function accessed by handler that dereferences global MAC pointer."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "TS 38.473 8.4.2.2",
      "location": "8.4.2.2",
      "text": "The gNB-CU initiates the procedure by sending a DL RRC MESSAGE TRANSFER message."
    },
    {
      "kind": "code",
      "source": "openair2/F1AP/f1ap_handlers.c",
      "location": "f1ap_handlers.c:43",
      "text": "static const f1ap_message_processing_t f1ap_messages_processing[][3] = { ... {DU_handle_DL_RRC_MESSAGE_TRANSFER, 0, 0}, /* DLRRCMessageTransfer */"
    },
    {
      "kind": "code",
      "source": "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c",
      "location": "mac_rrc_dl_handler.c:923",
      "text": "gNB_MAC_INST *mac = RC.nrmac[0];\\npthread_mutex_lock(&mac->sched_lock);"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {
      "from": "Attacker",
      "to": "gNB-CU",
      "message": "F1AP: DLRRCMessageTransfer (Initiating)"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-CU",
      "message": "f1ap_handle_message checks table"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-CU",
      "message": "Executes DU_handle_DL_RRC_MESSAGE_TRANSFER"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-CU",
      "message": "Calls dl_rrc_message_transfer"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-CU",
      "message": "Accesses RC.nrmac[0] (NULL on CU)"
    },
    {
      "from": "gNB-CU",
      "to": "gNB-CU",
      "message": "Crash"
    }
  ],
  "state_machine": {
    "states": [
      "Idle",
      "ReceivingPDU",
      "Dispatching",
      "ExecutingHandler",
      "Crashed"
    ],
    "transitions": [
      {
        "from": "Idle",
        "to": "ReceivingPDU",
        "on": "PDU Received"
      },
      {
        "from": "ReceivingPDU",
        "to": "Dispatching",
        "on": "Decode Success"
      },
      {
        "from": "Dispatching",
        "to": "ExecutingHandler",
        "on": "Handler Found in Table"
      },
      {
        "from": "ExecutingHandler",
        "to": "Crashed",
        "on": "Invalid Memory Access"
      }
    ]
  },
  "assumptions": [
    "gNB-CU does not initialize RC.nrmac or it is NULL"
  ],
  "questions": [
    "Does the SCTP layer filter messages based on stream/port?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "Message Direction Check",
      "result": "FAIL",
      "evidence": [
        "f1ap_handlers.c table allows calling DU handlers on CU node"
      ]
    },
    {
      "name": "Instance Role Check",
      "result": "FAIL",
      "evidence": [
        "f1ap_handle_message does not verify if instance matches handler role"
      ]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "The dispatch mechanism uses a shared table for both CU and DU roles and does not check if the receiving node is the correct type for the message handler."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "TS 38.473 8.4.2"
    ],
    "code_citations": [
      "openair2/F1AP/f1ap_handlers.c:43",
      "openair2/LAYER2/NR_MAC_gNB/mac_rrc_dl_handler.c:923"
    ],
    "expected_vs_observed": "Expected: CU should drop messages intended for DU (like DL RRC Message Transfer). Observed: CU executes DU handler and crashes."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "Critical vulnerability/bug where gNB-CU executes DU handlers when receiving DU-destined messages, leading to crash due to uninitialized MAC context.",
    "citations": {
      "spec": [
        "TS 38.473"
      ],
      "code": [
        "openair2/F1AP/f1ap_handlers.c"
      ]
    },
    "suggested_repro": [
      "Send F1AP DLRRCMessageTransfer to gNB-CU SCTP port"
    ],
    "risks": [
      "Denial of Service (Crash)"
    ],
    "next_steps": [
      "Implement node role check in f1ap_handle_message",
      "Separate dispatch tables for CU and DU"
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.